## Image creator

In [1]:
import shutil
import numpy as np
import cv2
import os

In [4]:
class ImgDFCreator:
    """
    This class creates a dataset of images, which are 64x64 px by default. Each picture 
    contains a solid red or pink circle on a differently shaded gray background. Circle covers
    10% - 60% of picture and is placed randomly somewhere inside. Also it's decided, that if
    a circle covers up to 30% of picture size - it's a small circle, otherwise - a big one.

    We also create folders (_clear_directories) to contain images and recreate them if they are already
    there. It is needed to create only 1 dataset per time with set parametres and number of examples.

    I wrote this code to get some pictures to train my CV model on. You can use this basic example 
    as a starting point and adjust it to creat images that fit your problem.
    """
    def __init__(self, width=64):
        self.width = width
        self.height = width
        self.circle_colors = [
            (0, 0, 255),    # Red (BGR)
            (255, 0, 255)   # Pink (BGR)
        ]
        self.gray_shades = [
            (50, 50, 50),   # Dark-grey
            (100, 100, 100),
            (150, 150, 150),
            (200, 200, 200),
            (230, 230, 230), # Light-grey
        ]
        self.small_circle_threshold = 0.3 * (self.width * self.height)
        self.directories = ['small_pink_circle', 'small_red_circle', 
                            'big_pink_circle', 'big_red_circle']

    def _clear_directories(self):
        for directory in self.directories:
            if os.path.exists(directory):
                shutil.rmtree(directory)
            os.makedirs(directory)

    def create_images(self, num_images):
        self._clear_directories()

        for i in range(num_images):
            circle_color = self.circle_colors[np.random.choice(len(self.circle_colors))]
            color_name = 'red' if circle_color == (0, 0, 255) else 'pink'
            background_color = self.gray_shades[np.random.choice(len(self.gray_shades))]
            image_np = np.full((self.height, self.width, 3), background_color, dtype=np.uint8)

            min_area = 0.1 * (self.width * self.height)
            max_area = 0.6 * (self.width * self.height)
            max_radius = np.sqrt(max_area / np.pi)
            min_radius = np.sqrt(min_area / np.pi)
            radius = np.random.randint(int(min_radius), int(max_radius))
            circle_area = np.pi * (radius ** 2)
            
            center_x, center_y = np.random.randint(radius, self.width - radius), np.random.randint(radius, self.height - radius)

            cv2.circle(image_np, (center_x, center_y), radius, circle_color, thickness=-1)

            if circle_area <= self.small_circle_threshold:
                if color_name == 'pink':
                    directory = 'small_pink_circle'
                else:
                    directory = 'small_red_circle'
            else:
                if color_name == 'pink':
                    directory = 'big_pink_circle'
                else:
                    directory = 'big_red_circle'
            
            filepath = f"{directory}/image_{i}.png"
            cv2.imwrite(filepath, image_np)
            print(f"Image saved to: {filepath}")



In [ ]:
creator = ImgDFCreator()
creator.create_images(10000)